In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pygame
import random

# Inisialisasi pygame untuk text-to-speech
pygame.mixer.init()

def load_dataset(data_path):
    X = []
    y = []
    color_labels = ['red', 'yellow', 'green', 'blue']
    
    for color in color_labels:
        color_path = os.path.join(data_path, color)
        for img_name in os.listdir(color_path):
            img_path = os.path.join(color_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                # Resize gambar untuk konsistensi
                img = cv2.resize(img, (100, 100))
                # Ekstrak warna dominan (kita gunakan rata-rata warna)
                avg_color = np.mean(img, axis=(0, 1))
                X.append(avg_color)
                y.append(color)
    
    return np.array(X), np.array(y)

# Path ke dataset Anda
dataset_path = 'C:\My Files\Kuliah\CAWU3\AI\Test AI 1\ColorDataset\ColorDataset'
X, y = load_dataset(dataset_path)

# Split dataset menjadi training dan testing (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\My Files\\Kuliah\\CAWU3\\AI\\Test AI 1\\ColorDataset\\ColorDataset\\red'

In [ ]:
def augment_image(img):
    augmented_images = []
    
    # Original image
    augmented_images.append(img)
    
    # 1. Flip Horizontal
    flipped_h = cv2.flip(img, 1)
    augmented_images.append(flipped_h)
    
    # 2. Flip Vertical
    flipped_v = cv2.flip(img, 0)
    augmented_images.append(flipped_v)
    
    # 3. Rotasi (10-30 derajat)
    angle = random.uniform(10, 30)
    rows, cols = img.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    rotated = cv2.warpAffine(img, M, (cols, rows))
    augmented_images.append(rotated)
    
    # 4. Perubahan Kecerahan
    value = random.uniform(0.7, 1.3)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:,:,2] = hsv[:,:,2]*value
    hsv[:,:,2][hsv[:,:,2]>255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    bright = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    augmented_images.append(bright)
    
    # 5. Perubahan Saturasi
    value = random.uniform(0.7, 1.3)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    saturated = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    augmented_images.append(saturated)
    
    # 6. Gaussian Blur (ringan)
    blurred = cv2.GaussianBlur(img, (5,5), 0)
    augmented_images.append(blurred)
    
    return augmented_images

def load_dataset_with_augmentation(data_path, augment=True):
    X = []
    y = []
    color_labels = ['red', 'yellow', 'green', 'blue']
    
    for color in color_labels:
        color_path = os.path.join(data_path, color)
        for img_name in os.listdir(color_path):
            img_path = os.path.join(color_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                # Resize gambar untuk konsistensi
                img = cv2.resize(img, (100, 100))
                
                if augment:
                    augmented_images = augment_image(img)
                else:
                    augmented_images = [img]
                
                for aug_img in augmented_images:
                    # Ekstrak warna dominan (kita gunakan rata-rata warna)
                    avg_color = np.mean(aug_img, axis=(0, 1))
                    X.append(avg_color)
                    y.append(color)
    
    return np.array(X), np.array(y)

In [3]:
# Path ke dataset Anda
dataset_path = 'C:\My Files\Kuliah\CAWU3\AI\Test AI 1\ColorDataset\ColorDataset'

# Load dataset dengan augmentasi
X, y = load_dataset_with_augmentation(dataset_path, augment=True)

# Split dataset menjadi training dan testing (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Jumlah data setelah augmentasi: {len(X)}")
print(f"Jumlah data training: {len(X_train)}")
print(f"Jumlah data testing: {len(X_test)}")

Jumlah data setelah augmentasi: 6664
Jumlah data training: 5331
Jumlah data testing: 1333


In [4]:
# Membuat dan melatih model k-NN
knn = KNeighborsClassifier(n_neighbors=5)  # Meningkatkan neighbors dari 3 ke 5
knn.fit(X_train, y_train)

# Evaluasi model
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model setelah Augmentasi: {accuracy*100:.2f}%")

Akurasi Model setelah Augmentasi: 93.62%


In [5]:
# Gunakan k-NN dengan parameter yang mendukung probabilitas
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')  # weights='distance' penting untuk probabilitas
knn.fit(X_train, y_train)

,n_neighbors,5
,weights,'distance'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [6]:
def detect_color_from_webcam(knn_model):
    cap = cv2.VideoCapture(0)
    
    # Parameter untuk ukuran bingkai (ROI)
    roi_size = 200
    show_roi = True
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Resize frame untuk pemrosesan lebih cepat
        frame = cv2.resize(frame, (640, 480))
        
        # Ambil bagian tengah frame
        h, w = frame.shape[:2]
        center_x, center_y = w//2, h//2
        roi = frame[center_y-roi_size//2:center_y+roi_size//2, 
                    center_x-roi_size//2:center_x+roi_size//2]
        
        # Hitung warna rata-rata ROI
        avg_color = np.mean(roi, axis=(0, 1))
        
        # Dapatkan probabilitas prediksi
        probabilities = knn_model.predict_proba([avg_color])[0]
        predicted_color = knn_model.classes_[np.argmax(probabilities)]
        
        # Tampilkan hasil
        cv2.rectangle(frame, (center_x-roi_size//2, center_y-roi_size//2),
                      (center_x+roi_size//2, center_y+roi_size//2), (255, 255, 255), 2)
        
        # Tampilkan warna prediksi utama
        cv2.putText(frame, f"Color: {predicted_color}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # Tampilkan persentase untuk semua warna
        y_offset = 60
        for i, (color, prob) in enumerate(zip(knn_model.classes_, probabilities)):
            # Tentukan warna teks berdasarkan warna yang ditampilkan
            text_color = (255, 255, 255)  # Putih default
            
            if color == "red":
                text_color = (0, 0, 255)  # Merah
            elif color == "yellow":
                text_color = (0, 255, 255)  # Kuning
            elif color == "green":
                text_color = (0, 255, 0)  # Hijau
            elif color == "blue":
                text_color = (255, 0, 0)  # Biru
                
            cv2.putText(frame, f"{color}: {prob*100:.1f}%", (10, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
            y_offset += 30
        
        # Tampilkan ROI yang diperbesar di sudut kanan atas
        if show_roi:
            roi_resized = cv2.resize(roi, (200, 200))
            frame[0:200, w-200:w] = roi_resized
            cv2.putText(frame, "ROI", (w-190, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        # Text-to-speech (akan berbicara ketika tombol 's' ditekan)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            text_to_speech(f"{predicted_color} {probabilities.max()*100:.0f} percent")
        elif key == ord('q'):
            break
            
        cv2.imshow('Color Detection', frame)
    
    cap.release()
    cv2.destroyAllWindows()
    
# Jalankan deteksi warna
detect_color_from_webcam(knn)